# 🎨 Video Colorizer (Google Colab)

*Colorize black-and-white videos using DeOldify — all within Google Colab!*


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](
https://colab.research.google.com/github/cyamahat/DeOldify_Video_Colorizer/blob/main/DeOldify_Video_Colorizer_Colab.ipynb)

**Updated: November 2025**


---

This notebook is intended as a tool to colorize gifs and **short videos**. If you are trying to convert longer video, you may hit the limit on processing space.

#### **Credits**

Colab notebook adapted from DeOldify's official (and archived) source.

## Verify Correct Runtime Settings

**IMPORTANT**

In the "Runtime" menu for the notebook window, select "Change runtime type." Ensure that the following are selected:
* Runtime Type = Python 3
* Hardware Accelerator = GPU


## Git clone and install DeOldify

In [2]:
!git clone https://github.com/jantic/DeOldify.git

Cloning into 'DeOldify'...
remote: Enumerating objects: 2620, done.
remote: Counting objects: 100% (583/583), done.
remote: Compressing objects: 100% (124/124), done.
remote: Total 2620 (delta 501), reused 459 (delta 459), pack-reused 2037 (from 1)
Receiving objects: 100% (2620/2620), 69.67 MiB | 15.17 MiB/s, done.
Resolving deltas: 100% (1210/1210), done.


In [3]:
cd DeOldify

/content/DeOldify


## Setup

In [4]:
#NOTE:  This must be the first call in order to work properly!
from deoldify import device
from deoldify.device_id import DeviceId
#choices:  CPU, GPU0...GPU7
device.set(device=DeviceId.GPU0)

import torch

if not torch.cuda.is_available():
    print('GPU not available.')

from os import path

In [5]:
!pip install -q -r requirements-colab.txt


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.0/182.0 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 631.4/631.4 kB 54.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 237.3/237.3 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 110.5 MB/s eta 0:00:00


In [6]:
import sys, os
import warnings
from contextlib import redirect_stdout, redirect_stderr

print("📦 Importing libraries... please wait.")

with open(os.devnull, "w") as f, redirect_stdout(f), redirect_stderr(f):
    import fastai
    from deoldify.visualize import *
    from pathlib import Path
    import torch
    torch.backends.cudnn.benchmark = True
    warnings.filterwarnings("ignore", category=UserWarning, message=".*?Your .*? set is empty.*?")

print("✅ Libraries imported successfully.")


INFO:numexpr.utils:NumExpr defaulting to 2 threads.


📦 Importing libraries... please wait.
NumExpr defaulting to 2 threads.
✅ Libraries imported successfully.


In [7]:
!mkdir 'models'
!wget https://data.deepai.org/deoldify/ColorizeVideo_gen.pth -O ./models/ColorizeVideo_gen.pth


mkdir: cannot create directory ‘models’: File exists
--2026-02-08 17:30:12--  https://data.deepai.org/deoldify/ColorizeVideo_gen.pth
Resolving data.deepai.org (data.deepai.org)... 84.17.38.232, 2400:52e0:1500::1181:1
Connecting to data.deepai.org (data.deepai.org)|84.17.38.232|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 874066230 (834M) [application/octet-stream]
Saving to: ‘./models/ColorizeVideo_gen.pth’

./models/ColorizeVi 100%[===================>] 833.57M  4.90MB/s    in 2m 50s  

2026-02-08 17:33:02 (4.90 MB/s) - ‘./models/ColorizeVideo_gen.pth’ saved [874066230/874066230]



In [8]:
import functools
import torch.serialization
from fastai.basic_train import Recorder
from fastai.callbacks.hooks import Hook
import fastai.callbacks.hooks as FCH
import torch.utils.hooks
import torch.nn.utils.spectral_norm

if not hasattr(torch, '_original_load_unpatched'):
    torch._original_load_unpatched = torch.load

def patched_torch_load(*args, **kwargs):
    current_patched_func_ref = torch.load
    torch.load = torch._original_load_unpatched

    try:
        if 'weights_only' not in kwargs:
            kwargs['weights_only'] = False
        return torch._original_load_unpatched(*args, **kwargs)
    finally:
        torch.load = patched_torch_load

if torch.load is not patched_torch_load:
    torch.load = patched_torch_load

colorizer = get_video_colorizer()

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResN

Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth


100%|██████████| 171M/171M [00:01<00:00, 131MB/s]
/usr/local/lib/python3.12/dist-packages/torch/nn/utils/weight_norm.py:144: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


## Instructions

### *source_url*
Type in a url hosting a video from YouTube, Imgur, Twitter, Reddit, Vimeo, etc.  Many sources work!  GIFs also work.

**Recommendation:** To use your own video, upload it to an online storage service such as **Google Drive** and make sure the link is **publicly accessible** (anyone with the link can view). Then, use that link in the notebook.


### *render_factor*
The default value of 21 has been carefully chosen and should work -ok- for most scenarios (but probably won't be the -best-). This determines resolution at which the color portion of the video is rendered. Lower resolution will render faster, and colors also tend to look more vibrant. Older and lower quality film in particular will generally benefit by lowering the render factor. Higher render factors are often better for higher quality videos and inconsistencies (flashy render) will generally be reduced, but the colors may get slightly washed out.



### How to Download a Copy
Simply right click on the displayed video and click "Save video as..."!



### Pro Tips
1. If a video takes a long time to render and you're wondering how well the frames will actually be colorized, you can preview how well the frames will be rendered at each render_factor by using the code at the bottom. Just stop the video rendering by hitting the stop button on the cell, then run that bottom cell under "See how well render_factor values perform on a frame here". It's not perfect and you may still need to experiment a bit especially when it comes to figuring out how to reduce frame inconsistency.  But it'll go a long way in narrowing down what actually works.
2. If videos are taking way too much time for your liking, running the Jupyter notebook VideoColorizer.ipynb on your own machine (with DeOldify installed) will generally be much faster (as long as you have the hardware for it).   
3. Longer videos (running multiple minutes) are going to have a rough time on Colabs. You'll be much better off using a local install of DeOldify instead in this case.



### Troubleshooting
The video player may wind up not showing up, in which case- make sure to wait for the Jupyter cell to complete processing first (the play button will stop spinning).  Then follow these alternative download instructions

1. In the menu to the left, click Files
2. If you don't see the 'DeOldify' folder, click "Refresh"
3. By default, rendered video will be in /DeOldify/video/result/

If a video you downloaded doesn't play, it's probably because the cell didn't complete processing and the video is in a half-finished state.

## Colorize!

In [ ]:
source_url = 'https://ia600109.us.archive.org/21/items/gravity-feynman/Conservation%20Law%20-%20Feynman.mp4' #@param {type:"string"}
render_factor = 21  #@param {type: "slider", min: 5, max: 40}
# watermarked = False #@param {type:"boolean"}
watermarked = False

if source_url is not None and source_url !='':
    video_path = colorizer.colorize_from_url(source_url, 'video.mp4', render_factor, watermarked=watermarked)
    show_video_in_notebook(video_path)
else:
    print('Provide a video url and try again.')

[generic] Extracting URL: https://ia600109.us.archive.org/21/items/gravity-feynman/Conservation%20Law%20-%20Feynman.mp4
[generic] Conservation Law - Feynman: Downloading webpage
[info] Conservation Law - Feynman: Downloading 1 format(s): mp4
[download] Destination: video/source/video.mp4
[download] 100% of  211.33MiB in 00:00:12 at 16.82MiB/s  


<div><progress max="100045" value="2900"></progress> 2.90% [2900/100045 07:12&lt;4:01:13]</div>

## See how well *render_factor* values perform on a frame here

In [ ]:
for i in range(10,40,2):
    colorizer.vis.plot_transformed_image('video/bwframes/video/01000.jpg', render_factor=i, display_render_factor=True, figsize=(8,8))